In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import glob
sys.path.append('/home/jleland/Coding/Projects/flopter')
import magopter as mg
import constants as c
import databases.magnum as mdb
import normalisation as nrm
import lputils as lp
import classes.ivdata as ivd
import filtering as filt
import scipy.signal as sig
import fitters as fts

In [12]:
folders = ['2018-05-01_Leland/', '2018-05-02_Leland/', '2018-05-03_Leland/',
           '2018-06-05_Leland/', '2018-06-06_Leland/', '2018-06-07_Leland/']
files = []
file_folders = []
for folder1 in folders:
    os.chdir(mg.Magopter.get_data_path() + folder1)
    files.extend(glob.glob('*.adc'))
    file_folders.extend([folder1] * len(glob.glob('*.adc')))
files.sort()

# file = files[286]
file = files[285]
ts_file = files[284]
folder = file_folders[-2]
print('"{}" \t\t "{}" \t\t "{}"'.format(folder, file, ts_file))

mp = lp.MagnumProbes()

"2018-06-07_Leland/" 		 "2018-06-07 14h 38m 33s TT_06564321037878221702.adc" 		 "2018-06-07 14h 33m 45s TT_06564319866763108260.adc"


In [13]:
magopter = mg.Magopter(folder, file, ts_filename=ts_file)

Start Time:  13:38:12
End Time:  13:38:52
Beam Down Timestamp:  6564321041100203608 1528375093.15
Regular Timestamp:  6564321037878221702 1528375092.399824


In [14]:
magopter.m_data.time = magopter.m_data.time + magopter._ADC_TIMER_OFFSET
magopter.m_data.data[magopter._VOLTAGE_CHANNEL] = magopter.m_data.data[magopter._VOLTAGE_CHANNEL] * 10

magopter.raw_time = magopter.m_data.time
magopter.raw_voltage = magopter.m_data.data[magopter._VOLTAGE_CHANNEL]
magopter.raw_current[0] = magopter.m_data.data[magopter._PROBE_CHANNEL_3]
magopter.raw_current[1] = magopter.m_data.data[magopter._PROBE_CHANNEL_4]

In [19]:
plt.figure()
plt.plot(magopter.raw_time, magopter.raw_voltage)
plt.plot(magopter.raw_time, magopter.raw_current[0])
# plt.plot(magopter.raw_time, magopter.raw_current[1])
plt.show()

In [21]:
data_dict = {
    'time': magopter.raw_time,
    'voltage': magopter.raw_voltage,
    'current': magopter.raw_current[0]
}
 
import pandas as pd

df = pd.DataFrame(data_dict)
print(df)

             time   voltage   current
0         1.00000 -6.752735 -0.168707
1         1.00001 -6.925882 -0.168707
2         1.00002 -7.099029 -0.151404
3         1.00003 -6.925882 -0.151404
4         1.00004 -7.099029 -0.160055
5         1.00005 -6.925882 -0.168707
6         1.00006 -7.185603 -0.134100
7         1.00007 -7.099029 -0.125449
8         1.00008 -7.185603 -0.134100
9         1.00009 -7.185603 -0.134100
10        1.00010 -7.272176 -0.108146
11        1.00011 -7.531897 -0.116797
12        1.00012 -7.185603 -0.108146
13        1.00013 -7.358750 -0.125449
14        1.00014 -7.531897 -0.116797
15        1.00015 -7.618470 -0.099494
16        1.00016 -7.358750 -0.099494
17        1.00017 -7.358750 -0.108146
18        1.00018 -7.964765 -0.073539
19        1.00019 -7.618470 -0.064887
20        1.00020 -7.791618 -0.064887
21        1.00021 -7.531897 -0.082191
22        1.00022 -8.137912 -0.064887
23        1.00023 -7.791618 -0.056236
24        1.00024 -8.137912 -0.082191
25        1.

In [22]:
df.to_csv('/home/jleland/Data/Magnum/raw_lp_trace.csv')